In [375]:
import pandas as pd
import numpy as np

selected_features = ['pre_tax_profit_margin', 'net_profit_margin', 'asset_turnover', 'roe',
       'return_on_tangible_equity', 'roa', 'roi', 'book_value_per_share',
       'operating_cash_flow_per_share', 'free_cash_flow_per_share',
       'cash_on_hand', 'total_current_assets', 'total_long_term_assets',
       'total_assets', 'total_current_liabilities',
       'total_long_term_liabilities', 'total_liabilities',
       'retained_earnings_accumulated_deficit', 'total_share_holder_equity',
       'total_liabilities_share_holders_equity', 'revenue', 'gross_profit',
       'operating_income', 'pre_tax_income', 'income_after_taxes',
       'income_from_continuous_operations', 'net_income',
       'basic_shares_outstanding', 'shares_outstanding',
       'eps_basic_net_earnings_per_share', 'eps_earnings_per_share_diluted',
       'net_income_loss', 'total_depreciation_amortization_cash_flow',
       'other_non_cash_items', 'total_non_cash_items',
       'total_change_in_assets_liabilities',
       'cash_flow_from_operating_activities',
       'cash_flow_from_investing_activities',
       'cash_flow_from_financial_activities', 'net_cash_flow']

selected_features = ['comprehensive_income_3_quartes_gain',
 'total_share_holder_equity_3_quartes_gain',
 'gross_profit_3_quartes_gain',
 'operating_income_3_quartes_gain',
 'pre_tax_income_3_quartes_gain',
 'market_cap',
 'total_share_holder_equity_2_quartes_gain',
 'comprehensive_income_5_quartes_gain',
 'ebit_3_quartes_gain',
 'eps_earnings_per_share_diluted_3_quartes_gain',
 'eps_basic_net_earnings_per_share_3_quartes_gain',
 'book_value_per_share_3_quartes_gain',
 'revenue_3_quartes_gain',
 'pre_tax_profit_margin_3_quartes_gain',
 'comprehensive_income_2_quartes_gain',
 'income_after_taxes_3_quartes_gain',
 'book_value_per_share_2_quartes_gain',
 'income_from_continuous_operations_3_quartes_gain',
 'pre_tax_profit_margin_5_quartes_gain',
 'net_income_loss_3_quartes_gain',
 'gross_profit_2_quartes_gain',
 'net_income_3_quartes_gain',
 'shares_outstanding_3_quartes_gain',
 'operating_income_5_quartes_gain',
 'pre_tax_income_5_quartes_gain',
 'debt_equity_ratio_3_quartes_gain',
 'ebitda_3_quartes_gain',
 'roi_3_quartes_gain',
 'comprehensive_income',
 'roa_5_quartes_gain',
 'ebit_margin_3_quartes_gain',
 'roa_3_quartes_gain',
 'operating_margin_3_quartes_gain',
 'comprehensive_income_21_quartes_gain',
 'eps_basic_net_earnings_per_share_5_quartes_gain',
 'shares_outstanding_2_quartes_gain',
 'revenue_2_quartes_gain',
 'roe_3_quartes_gain',
 'asset_turnover_3_quartes_gain',
 'income_from_continuous_operations_5_quartes_gain',
 'net_income_5_quartes_gain',
 'income_after_taxes_5_quartes_gain',
 'eps_earnings_per_share_diluted_5_quartes_gain']


selected_features = ['net_property_plant_equipment',
 'total_long_term_assets',
 'total_liabilities_share_holders_equity',
 'total_assets',
 'total_long_term_liabilities',
 'total_liabilities',
 'book_value_per_share',
 'long_term_debt',
 'total_current_liabilities',
 'total_share_holder_equity',
 'retained_earnings_accumulated_deficit',
 'operating_income_3_quartes_gain',
 'total_depreciation_amortization_cash_flow',
 'total_long_term_liabilities_21_quartes_gain',
 'pre_tax_income_5_quartes_gain',
 'operating_expenses',
 'net_property_plant_equipment_21_quartes_gain',
 'pre_tax_income_3_quartes_gain',
 'total_long_term_assets_21_quartes_gain',
 'asset_turnover',
 'total_long_term_assets_13_quartes_gain',
 'total_liabilities_21_quartes_gain',
 'net_property_plant_equipment_8_quartes_gain',
 'total_assets_21_quartes_gain',
 'total_liabilities_share_holders_equity_21_quartes_gain',
 'ebit_3_quartes_gain',
 'pre_tax_profit_margin_5_quartes_gain',
 'other_long_term_assets',
 'roi_5_quartes_gain',
 'income_from_continuous_operations_3_quartes_gain',
 'operating_income_5_quartes_gain',
 'gross_profit_3_quartes_gain',
 'net_property_plant_equipment_13_quartes_gain',
 'income_after_taxes_3_quartes_gain',
 'roa_5_quartes_gain',
 'pre_tax_profit_margin_3_quartes_gain',
 'roi_8_quartes_gain',
 'total_share_holder_equity_2_quartes_gain',
 'selling_general_administrative_expenses',
 'roi_percentile_in_index',
 'revenue_3_quartes_gain',
 'pre_tax_income_8_quartes_gain',
 'pre_tax_profit_margin_8_quartes_gain',
 'goodwill_intangible_assets_total']


In [377]:
df = pd.read_csv("sp500/data_xy.csv")
df = df.drop(columns=["quarter", "ticker"])
df = df.dropna(subset='log_return_q3')
df = df.fillna(0.0)
Y = df['class_2_q3']
X = df[selected_features]
X.head()


,net_property_plant_equipment,total_long_term_assets,total_liabilities_share_holders_equity,total_assets,total_long_term_liabilities,total_liabilities,book_value_per_share,long_term_debt,total_current_liabilities,total_share_holder_equity,...,roa_5_quartes_gain,pre_tax_profit_margin_3_quartes_gain,roi_8_quartes_gain,total_share_holder_equity_2_quartes_gain,selling_general_administrative_expenses,roi_percentile_in_index,revenue_3_quartes_gain,pre_tax_income_8_quartes_gain,pre_tax_profit_margin_8_quartes_gain,goodwill_intangible_assets_total
788,957.000,3365.000,9100.00,9100.00,3373.000,6290.00,8.1214,2177.000,2917.000,2810.000,...,0.0,0.0,0.0,0.000,456.000,70.129599,0.0,0.0,0.0,1912.000
789,4768.000,33505.000,75183.00,75183.00,6670.000,27392.00,1.8634,0.000,20722.000,47791.000,...,0.0,0.0,0.0,8443.000,1571.000,94.661447,0.0,0.0,0.0,1083.000
790,7939.787,37926.200,57373.62,57373.62,20307.030,35890.89,13.8973,12910.680,15583.870,21482.720,...,0.0,0.0,0.0,470.380,2673.277,45.983617,0.0,0.0,0.0,28454.210
791,0.000,13360.900,17701.40,17701.40,3274.436,12983.49,10.6493,2615.670,9709.051,4717.910,...,0.0,0.0,0.0,339.153,0.000,34.530147,0.0,0.0,0.0,0.000
792,659.569,3271.628,12835.25,12835.25,2992.926,9560.53,4.7458,1.445,6567.604,3274.723,...,0.0,0.0,0.0,0.000,1131.118,100.000000,0.0,0.0,0.0,841.234


In [ ]:
Y.head()

788    1.0
789    1.0
790    0.0
791    1.0
792    1.0
Name: class_2_q3, dtype: float64

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from tqdm.auto import tqdm
import numpy as np

class GridSearchCVProgressBar(GridSearchCV):
    pass

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from scipy.stats import randint, uniform


def parse_quarter(x):
    year = 2010 + (x // 4)
    q = (x % 4) + 1
    return f"{year}Q{q}"


def train_test_split_balanced(X, Y, column, test_size=0.2, random_state=42):
    XY = X.join(Y)
    class_0 = XY[XY[column] == 0]
    class_1 = XY[XY[column] == 1]
    n_samples = min(len(class_0), len(class_1))
    class_0_sampled = class_0.sample(n=n_samples, random_state=random_state)
    class_1_sampled = class_1.sample(n=n_samples, random_state=random_state)
    balanced_df = pd.concat([class_0_sampled, class_1_sampled])
    balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
    X = balanced_df.drop(columns=[column])
    y = balanced_df[column]
    
    return train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )



import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns

def train_and_evaluate_classifiers(X, y, test_size=0.2, random_state=42):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split_balanced(
        X, y, 'class_2_q3', test_size=test_size, random_state=random_state#, stratify=y
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)    
    # Define classifiers with their parameter grids
    
    classifiers = {
        'RandomForestClassifier': (
            RandomForestClassifier(random_state=random_state),
            {
                'n_estimators': randint(500, 1200),  # Integer distribution
                'max_depth': [None] + list(randint(10, 40).rvs(4)),
                #_min_samples_split': randint(2, 10),
                #_min_samples_leaf': randint(1, 5),
                #_class_weight': ['balanced', 'balanced_subsample', class_weight_dict, None],
                'max_features': ['sqrt', 'log2']
            }
        ),
         'XGBoost': (
            XGBClassifier(random_state=random_state),
            {
                'n_estimators': randint(100, 1200),
                'max_depth': randint(3, 10),
                'learning_rate': uniform(0.01, 0.09),  # Continuous from 0.01 to 0.1
                'min_child_weight': randint(1, 6),
                'gamma': uniform(0, 0.3),
                'subsample': uniform(0.5, 0.5),  # Continuous from 0.5 to 1.0
                'colsample_bytree': uniform(0.6, 0.4),
            }
        ),
    }
    
    # Store results
    results = {}
    
    # Train and evaluate each classifier
    for name, (classifier, param_grid) in classifiers.items():
        print(f"\nTraining {name}...")
        
        # Create pipeline
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', classifier)
        ])
        
        # Perform grid search
        grid_search = RandomizedSearchCV(
            classifier,
            param_grid,
            cv=5,
            scoring='f1',
            n_jobs=-1,
            verbose=4,
            n_iter=15,
            random_state=random_state
        )
        
        # Fit the model
        grid_search.fit(X_train_scaled, y_train)
        
        # Make predictions
        y_pred = grid_search.predict(X_test_scaled)
        
        # Store results
        results[name] = {
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'test_accuracy': accuracy_score(y_test, y_pred),
            'classification_report': classification_report(y_test, y_pred),
            'confusion_matrix': confusion_matrix(y_test, y_pred),
            'predictions': y_pred
        }
        
        print(f"\nBest parameters for {name}:")
        print(results[name]['best_params'])
        print(f"\nTest accuracy for {name}: {results[name]['test_accuracy']:.4f}")
        print(f"\nClassification Report for {name}:")
        print(results[name]['classification_report'])
    
    return results, X_test, y_test

def plot_results(results, X_test, y_test):
    # Plot accuracy comparison
    plt.figure(figsize=(12, 6))
    accuracies = [results[name]['test_accuracy'] for name in results.keys()]
    plt.bar(results.keys(), accuracies)
    plt.title('Model Accuracy Comparison')
    plt.xlabel('Classifier')
    plt.ylabel('Accuracy')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Plot confusion matrices with clear winner/loser labels
    n_classifiers = len(results)
    n_rows = (n_classifiers + 2) // 3  # Ceiling division to get enough rows
    fig, axes = plt.subplots(n_rows, 3, figsize=(15, 5*n_rows))
    axes = axes.ravel()
    
    classes = ['Loser (0)', 'Winner (1)']
    
    for idx, (name, result) in enumerate(results.items()):
        cm = result['confusion_matrix']
        
        # Create confusion matrix plot with seaborn
        sns.heatmap(
            cm,
            annot=True,
            fmt='d',
            ax=axes[idx],
            cmap='Blues',
            xticklabels=classes,
            yticklabels=classes
        )
        
        axes[idx].set_title(f'{name} Confusion Matrix')
        axes[idx].set_xlabel('Predicted')
        axes[idx].set_ylabel('Actual')
        
        # Add text annotations explaining the quadrants
        axes[idx].text(-0.2, -0.1, 
                      'TN: Correctly\nPredicted Losers', 
                      transform=axes[idx].transAxes,
                      fontsize=8)
        axes[idx].text(0.8, -0.1, 
                      'FP: Losers Predicted\nas Winners', 
                      transform=axes[idx].transAxes,
                      fontsize=8)
        axes[idx].text(-0.2, 0.9, 
                      'FN: Winners Predicted\nas Losers', 
                      transform=axes[idx].transAxes,
                      fontsize=8)
        axes[idx].text(0.8, 0.9, 
                      'TP: Correctly\nPredicted Winners', 
                      transform=axes[idx].transAxes,
                      fontsize=8)
    
    # Hide any unused subplots
    for idx in range(len(results), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout(pad=3.0)
    plt.show()
    
    # Add a summary of key metrics for each class
    for name, result in results.items():
        print(f"\nDetailed metrics for {name}:")
        report_dict = classification_report(
            y_test, 
            result['predictions'], 
            target_names=['Loser (0)', 'Winner (1)'], 
            output_dict=True
        )
        
        # Create a prettier format for the metrics
        metrics_df = pd.DataFrame({
            'Class': ['Loser (0)', 'Winner (1)'],
            'Precision': [report_dict['Loser (0)']['precision'], 
                         report_dict['Winner (1)']['precision']],
            'Recall': [report_dict['Loser (0)']['recall'], 
                      report_dict['Winner (1)']['recall']],
            'F1-Score': [report_dict['Loser (0)']['f1-score'], 
                        report_dict['Winner (1)']['f1-score']],
            'Support': [report_dict['Loser (0)']['support'], 
                       report_dict['Winner (1)']['support']]
        }).set_index('Class')
        
        print(metrics_df.round(3))
        print("\nConfusion Matrix Interpretation:")
        cm = result['confusion_matrix']
        print(f"True Negatives (Correctly Predicted Losers): {cm[0,0]}")
        print(f"False Positives (Losers Predicted as Winners): {cm[0,1]}")
        print(f"False Negatives (Winners Predicted as Losers): {cm[1,0]}")
        print(f"True Positives (Correctly Predicted Winners): {cm[1,1]}")


results, X_test, y_test = train_and_evaluate_classifiers(X, Y)
plot_results(results, X_test, y_test)


Training RandomForestClassifier...
Fitting 5 folds for each of 15 candidates, totalling 75 fits


[CV 2/5] END max_depth=24, max_features=sqrt, n_estimators=520;, score=0.613 total time=  56.9s
[CV 1/5] END max_depth=24, max_features=sqrt, n_estimators=520;, score=0.625 total time=  57.0s
[CV 3/5] END max_depth=24, max_features=sqrt, n_estimators=520;, score=0.620 total time=  58.9s
[CV 4/5] END max_depth=22, max_features=sqrt, n_estimators=770;, score=0.636 total time= 1.4min
[CV 2/5] END max_depth=22, max_features=sqrt, n_estimators=770;, score=0.617 total time= 1.4min
[CV 1/5] END max_depth=22, max_features=sqrt, n_estimators=770;, score=0.625 total time= 1.4min
[CV 3/5] END max_depth=22, max_features=sqrt, n_estimators=770;, score=0.623 total time= 1.4min
[CV 5/5] END max_depth=22, max_features=sqrt, n_estimators=770;, score=0.622 total time= 1.4min
[CV 4/5] END max_depth=24, max_features=sqrt, n_estimators=520;, score=0.631 total time=  57.7s
[CV 5/5] END max_depth=24, max_features=sqrt, n_estimators=520;, score=0.615 total time=  59.0s
[CV 1/5] END max_depth=12, max_features=

/Users/micmur/PAD/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 4/5] END max_depth=37, max_features=log2, n_estimators=843;, score=0.632 total time= 1.3min
[CV 4/5] END max_depth=24, max_features=None, n_estimators=587;, score=0.640 total time= 7.0min
[CV 5/5] END max_depth=24, max_features=None, n_estimators=587;, score=0.617 total time= 7.0min
[CV 5/5] END max_depth=37, max_features=log2, n_estimators=843;, score=0.628 total time= 1.3min
[CV 1/5] END max_depth=22, max_features=log2, n_estimators=885;, score=0.623 total time= 1.4min
[CV 2/5] END max_depth=22, max_features=log2, n_estimators=885;, score=0.616 total time= 1.4min
[CV 3/5] END max_depth=22, max_features=log2, n_estimators=885;, score=0.613 total time= 1.4min
[CV 4/5] END max_depth=22, max_features=log2, n_estimators=885;, score=0.632 total time= 1.4min
[CV 5/5] END max_depth=22, max_features=log2, n_estimators=885;, score=0.621 total time= 1.4min
[CV 1/5] END max_depth=22, max_features=sqrt, n_estimators=660;, score=0.623 total time= 1.2min
[CV 2/5] END max_depth=22, max_features=

KeyboardInterrupt: 

26387